### Importing libraries

In [ ]:
import darts
import torch
import pandas as pd 
import numpy as np
import warnings
import matplotlib.pyplot as plt 
from pandas.plotting import autocorrelation_plot
from darts.timeseries import TimeSeries
from darts.utils.timeseries_generation import gaussian_timeseries
from darts.utils.timeseries_generation import random_walk_timeseries
from darts.dataprocessing.transformers.scaler import Scaler
from darts.utils.statistics import plot_hist

# jupyter notebook settings
plt.style.use('seaborn')
warnings.filterwarnings("ignore")
np.random.seed(42)    # reproducibility
torch.manual_seed(42) # reproducibility
random_state = 42
%matplotlib inline

### Generate Dataset and implement statistical analysis

In [ ]:
dataset = random_walk_timeseries(start = pd.Timestamp(year=2000, month=1, day=1),
                                 end   = pd.Timestamp(year=2001, month=1, day=1))
plt.figure(figsize=(8, 2))
dataset.plot(label='Time Series')
plt.title('Random Walk Time Series')
plt.ylabel('Amplitude')
plt.xlabel('Date')
plt.legend()
print(len(dataset))

In [ ]:
plt.figure(figsize=(8, 4))
autocorrelation_plot(dataset.pd_dataframe())

In [ ]:
plot_hist(dataset, density=True, fig_size=(10,4))

In [ ]:
# scaled time series
scaler = Scaler()
scaled_series = scaler.fit_transform(dataset.copy())

In [ ]:
train_size = int(0.7 * len(scaled_series)) # 70% to train
valid_size = int(0.2 * len(scaled_series)) # 20% to validation
test_size  = int(0.1 * len(scaled_series)) # 10% to test

train = scaled_series[ : train_size]
val   = scaled_series[train_size : train_size + valid_size]
test  = scaled_series[train_size + valid_size : train_size + valid_size + test_size]

plt.figure(figsize=(8, 2))
train.plot(label='Train series')
val.plot(label='Validation series')
test.plot(label='Test series')
plt.title('Random Walk Time Series')
plt.ylabel('Amplitude')
plt.xlabel('Date')
plt.legend()

#### Models implementation

In [ ]:
# General parameters
input_size  = 2 * len(test)
output_size = len(test)
n_epochs    = 10 # number of epochs to ML-based models
batch_size  = 32

if torch.cuda.is_available():
    pl_trainer_kwargs = {'accelerator'      : 'gpu',
                         'devices'          : -1,
                         'auto_select_gpus' : True}
else:
    pl_trainer_kwargs = {'accelerator' : 'cpu'}

In [ ]:
from darts.models.forecasting.regression_model import RegressionModel
from darts.models.forecasting.auto_arima import AutoARIMA
from darts.models.forecasting.nbeats import NBEATSModel
from darts.models.forecasting.nhits import NHiTSModel
from darts.models.forecasting.transformer_model import TransformerModel

models = {'RegressionModel': RegressionModel(lags = input_size,
                                             output_chunk_length = output_size),
          'ARIMA': AutoARIMA(random_state = random_state),
          'NBEATS': NBEATSModel(input_chunk_length = input_size,
                                output_chunk_length = output_size,
                                n_epochs = n_epochs,
                                batch_size = batch_size,
                                pl_trainer_kwargs = pl_trainer_kwargs,
                                generic_architecture = True,
                                random_state = random_state),
          'NHiTS': NHiTSModel(input_chunk_length = input_size,
                              output_chunk_length = output_size,
                              n_epochs = n_epochs,
                              batch_size = batch_size,
                              pl_trainer_kwargs = pl_trainer_kwargs),
          'TransformerModel': TransformerModel(input_chunk_length = input_size,
                                               output_chunk_length = output_size,
                                               pl_trainer_kwargs = pl_trainer_kwargs,
                                               random_state = random_state)}

In [ ]:
fitted_models = {}
for name, model in models.items():
    fitted_models[name] = model.fit(train)
